Clase 9 y 10
======

Objetivos

* Algortimo de alineamiento de Needleman-Wunsh

Anuncios
----------

Felicitaciones a quienes ya han entregado la tarea optativa de la clase 8. Para no la han entregado, recuerden que hay plazo hasta el próximo lunes (antes del comienzo de la clase) para hacerlo. 

Algortimo de alineamiento
===========


En la clase anterior aprendimos a evaluar la bondad de un alineamiento entre dos secuencias a través de la medición de su score. Si tenemos dos alineamientos, elegimos el que tenga el mayor score. Esto implica, que el dentro de todos los alineamientos posibles entre dos secuencias, el mejor es aquel que maximiza el score.

Para esto utilizaremos el algoritmo de Needleman-Wuns, el cual, dado una matriz de substitución (por ejemplo BLOSUM50) encuentra el alineamiento con el maximo score posible.

Antes de ver el algoritmo de Needleman-Wunsh, veamos maneras convenientes de representar los alineamientos.


Representación matricial del alineamiento de dos secuencias
------------------------------

Exploremos la forma de representar alineamientos a través de ejemplos. Las siguientes secuencias:

```
x=AETGGW
y=AEEH
```

Pueden ser alineadas como

```
AETGGW
A-E-EH
```

o

```
AETGGW
AEE--H
```

o de varias otras formas. Para expresar todos los posibles alineamientos de manera compacta podemos hacer uso de la siguiente representación matricial

<img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase9_matrixAlingment1.png" alt="Drawing" style="width: 300px;" />


¿Cómo podemos representar alineamientos que comienzan con un gap? Por ejemplo:

```
AETGGW
-AEE-H
```

o

```
-AETGGW
AE-E--H
```

Para ésto podemos agregar la siguiente modificación a nuesta matriz de alineamientos:


<img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase9_alignmentMatrix2.png" alt="Drawing" style="width: 300px;" />


Ejercicio 1
-----------

Representa en forma matricial los siguientes alienamientos:

1.
```
GTAAETGGW
-WTAE-E-H
```
2.
```
-ETGGTAGW
E-E--WTAH
```


Ejercicio 2
-----------

Para verificar que entendimos como ésto funciona, ahora hagamos el proceso en sentido inverso. Dadas las siguientes matrices, representa el alineamiento como una secuencia letra a letra:

1.
<img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase9_example1.png" alt="Drawing" style="width: 300px;" />

2.
<img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase9_example2.png" alt="Drawing" style="width: 300px;" />



¿De qué sirve ésto? En la misma matríz podemos anotar los scores de los alineamientos parciales
------

Supongamos que queremos encontrar el mejor alineamineto entre $x="HEAGAWGHEE"$ e $y="PAWHEAE"$, utilizando la matriz BLOSUM50 (scores para match/mismatch) y $d=-8$ (score para gaps). En este caso podemos escribir todas la alineaciones posibles y los scores de los mejores alineamientos parciales como la matriz:

<img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase7_exampleAlignment.jpg" alt="Drawing" style="width: 500px;" />

En donde la coordenada $(i,j)$ corresponde al mejor score entre las secuencias $x_{1...i}$ y $y_{1...j}$. Calculamos el valor de cada coordenadas en base a sus valores adyacentes:

<img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase7_trace.jpg" alt="Drawing" style="width: 400px"/>

De estas tres opciones, se selecciona el score que hasta ese momento sea el más alto:

<img src="https://raw.githubusercontent.com/mrivas/Bioinformatica/master/clase7_selection.jpg" alt="Drawing" style="width: 400px;"/>

Veamos el ejemplo paso a paso a través de los siguientes ejercicios.


Ejercicio 4
----------

Dada las secuencias $x="HEAGAWGHEE"$ e $y="PAWHEAE"$, escribe código para:

1. Construir la matriz con los scores de cada secuencia.
2. Computar la matriz de los los mejores alineamientos parciales.
3. Trazar la suceción de alineamientos parciales con el mayor score total.
4. Escribe una función ```align(seq1,seq2)``` que tome como argumentos las secuencias ```seq1``` y ```seq2```, y entrgue como resultado su mejor alineamiento. 

In [2]:
# 1

seq1="HEAGAWGHEE"
seq2="PAWHEAE"


seq1=createList(seq1)
seq2=createList(seq2)

scoreMatrix=np.zeros((len(seq2),len(seq1) ) )  

for i in range(len(seq2)):
    for j in range(len(seq1)):
        pair = (seq2[i],seq1[j])
        if not pair in MatrixInfo.blosum50:
            pair = tuple(reversed(pair))
        scoreMatrix[i,j]=MatrixInfo.blosum50[pair]

scoreMatrix=pd.DataFrame(scoreMatrix,index=seq2,columns=seq1) 
scoreMatrix

,H,E,A,G,A,W,G,H,E,E
P,-2.0,-1.0,-1.0,-2.0,-1.0,-4.0,-2.0,-2.0,-1.0,-1.0
A,-2.0,-1.0,5.0,0.0,5.0,-3.0,0.0,-2.0,-1.0,-1.0
W,-3.0,-3.0,-3.0,-3.0,-3.0,15.0,-3.0,-3.0,-3.0,-3.0
H,10.0,0.0,-2.0,-2.0,-2.0,-3.0,-2.0,10.0,0.0,0.0
E,0.0,6.0,-1.0,-3.0,-1.0,-3.0,-3.0,0.0,6.0,6.0
A,-2.0,-1.0,5.0,0.0,5.0,-3.0,0.0,-2.0,-1.0,-1.0
E,0.0,6.0,-1.0,-3.0,-1.0,-3.0,-3.0,0.0,6.0,6.0


In [3]:
# 2 y 3
seq1="-HEAGAWGHEE"
seq2="-PAWHEAE"

seq1=createList(seq1)
seq2=createList(seq2)

d=8
traceMatrix=np.zeros((len(seq2),len(seq1)) ) 
alignMatrix=np.zeros((len(seq2),len(seq1)) )  

# Set first column
alignMatrix[:,0] = -d*np.array(range( len(seq2) ))
# Set first row

alignMatrix[0,:] = -d*np.array(range( len(seq1) ))

for i in range(1,len(seq2)):
    for j in range(1,len(seq1)):
        value_i=seq2[i]
        value_j=seq1[j]
        pair = (value_i,value_j)
        if not pair in MatrixInfo.blosum50:
            pair = tuple(reversed(pair))
        score=MatrixInfo.blosum50[pair]
        alignMatrix[i,j] = np.max([alignMatrix[i,j-1]-d,alignMatrix[i-1,j-1]+score,alignMatrix[i-1,j]-d,])
        indexMax=       np.argmax([alignMatrix[i,j-1]-d,alignMatrix[i-1,j-1]+score,alignMatrix[i-1,j]-d])
        traceMatrix[i,j] = indexMax
traceMatrix = pd.DataFrame(traceMatrix,index=seq2,columns=seq1) 
alignMatrix = pd.DataFrame(alignMatrix,index=seq2,columns=seq1) 


In [4]:
alignMatrix

,-,H,E,A,G,A,W,G,H,E,E
-,0.0,-8.0,-16.0,-24.0,-32.0,-40.0,-48.0,-56.0,-64.0,-72.0,-80.0
P,-8.0,-2.0,-9.0,-17.0,-25.0,-33.0,-41.0,-49.0,-57.0,-65.0,-73.0
A,-16.0,-10.0,-3.0,-4.0,-12.0,-20.0,-28.0,-36.0,-44.0,-52.0,-60.0
W,-24.0,-18.0,-11.0,-6.0,-7.0,-15.0,-5.0,-13.0,-21.0,-29.0,-37.0
H,-32.0,-14.0,-18.0,-13.0,-8.0,-9.0,-13.0,-7.0,-3.0,-11.0,-19.0
E,-40.0,-22.0,-8.0,-16.0,-16.0,-9.0,-12.0,-15.0,-7.0,3.0,-5.0
A,-48.0,-30.0,-16.0,-3.0,-11.0,-11.0,-12.0,-12.0,-15.0,-5.0,2.0
E,-56.0,-38.0,-24.0,-11.0,-6.0,-12.0,-14.0,-15.0,-12.0,-9.0,1.0


In [5]:
traceMatrix

,-,H,E,A,G,A,W,G,H,E,E
-,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
P,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
W,0.0,2.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
H,0.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0
E,0.0,2.0,1.0,0.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0
A,0.0,2.0,2.0,1.0,0.0,1.0,1.0,1.0,2.0,2.0,1.0
E,0.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
